### Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Scraping text and coverting it in required format.###

In [2]:
base_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
index = requests.get(base_url).text
soup=BeautifulSoup(index)

table = soup.find('table',{'class':'wikitable sortable'}).get_text()
table=table.split('\n')

df=pd.DataFrame(table)
df.rename(columns={0: 'text'},inplace=True)
df=df[df.text!='']
df=df.reset_index()
df.drop('index',axis=1,inplace=True)
df.head()

,text
0,Postcode
1,Borough
2,Neighbourhood
3,M1A
4,Not assigned


Now converting this data in required format.

In [3]:
postcodes = []
boroughs = []
neighborhoods = []

for i in range(3,len(df)):
    if i%3==0:
        postcodes.append(df.iloc[i][0])
    if i%3==1:
        boroughs.append(df.iloc[i][0])
    if i%3==2:
        neighborhoods.append(df.iloc[i][0])

toronto_df = pd.DataFrame({'PostalCode': postcodes,'Borough': boroughs,'Neighborhood': neighborhoods})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Now removing rows where Borough='Not assigned'.

In [4]:
toronto_df = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


As shown, there is one record where Neighbourhood='Not Assigned'

In [5]:
toronto_df[toronto_df.Neighborhood=='Not assigned']

,PostalCode,Borough,Neighborhood


Assigning Neighborhood = Borough where Neighbourhood='Not Assigned'

In [6]:
for index, row in toronto_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

toronto_df[toronto_df.Borough=='Queen\'s Park']

,PostalCode,Borough,Neighborhood


Grouping the data as required.

In [7]:
grouped_df=toronto_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda k: ", ".join(k))
grouped_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Showing the shape of the data-frame.

In [8]:
print(grouped_df.shape)

(103, 3)


## 2.Getting geographic coordinates and adding them to the dataframe.

In [9]:
coords = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
coords=coords.rename(columns={"Postal Code": "PostalCode"})
df=grouped_df.merge(coords, on='PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Clustering the neighborhoods.

Visualizing the neighborhoods:

In [11]:
latitudes=df['Latitude'].to_list()
longitudes=df['Longitude'].to_list()
toronto_map = folium.Map(location=[43.6532,-79.3832], zoom_start=11)
toronto_map

toronto_latitude=43.6532
toronto_longitude=-79.3832

neighborhoods = folium.map.FeatureGroup()
for lat, lng, in zip(df.Latitude, df.Longitude):
    neighborhoods.add_child(
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6))

toronto_map.add_child(neighborhoods)

As done in the New York lab, I'll only explore data that has the word "Toronto" in the Borough column.

In [12]:
df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [13]:
toronto_boroughs=['East Toronto','Central Toronto','Downtown Toronto','West Toronto']
df=df[df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [14]:
latitudes=df['Latitude'].to_list()
longitudes=df['Longitude'].to_list()

toronto_map = folium.Map(location=[43.6532,-79.3832], zoom_start=11)
toronto_map

neighborhoods = folium.map.FeatureGroup()
for lat, lng, in zip(df.Latitude, df.Longitude):
    neighborhoods.add_child(
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6))

toronto_map.add_child(neighborhoods)

Foursquare Credentials are crossed.

In [15]:
CLIENT_ID = 'XXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Exploring the neighborhoods:
Getting venues near each neighborhood using foursquare API.

In [16]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

toronto_venues.head()

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Checking the number of values returned for each neighborhood.

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,79,79,79,79,79,79
"Chinatown, Grange Park, Kensington Market",85,85,85,85,85,85
Christie,18,18,18,18,18,18


Number of unique categories that can be curated from these values:

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


Now analyzing each neighborhood. <br>
One hot encoding:

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
col_at_beg=['Neighborhood']
toronto_onehot = toronto_onehot[[c for c in col_at_beg if c in toronto_onehot]+[c for c in toronto_onehot if c not in col_at_beg]]

print('Shape of toronto_onehot dataframe='+str(toronto_onehot.shape))
toronto_onehot.head()


Shape of toronto_onehot dataframe=(1705, 237)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(39, 237)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,...,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.012658,0.000000,0.00,...,0.00,0.012658,0.000000,0.012658,0.000000,0.012658,0.000000,0.000000,0.00,0.012658
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00,0.035294,0.000000,0.000000,0.047059,0.011765,0.000000,0.000000,0.00,0.000000
8,Christie,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011905,0.0000,0.0000,0.000,0.0000,0.000,0.011905,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.011905,0.00,0.011905


Getting top 5 most common venues for each neighborhood and putting them into a dataframe.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Bar,Thai Restaurant
1,Berczy Park,Coffee Shop,Café,Bakery,Seafood Restaurant,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Performing Arts Venue,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Brewery,Skate Park,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry


Running k-means to cluster the neighborhoods into 10 clusters.

In [22]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([8, 8, 8, 0, 4, 8, 2, 8, 8, 2])

Creating a new dataframe that includes the clusters as well as the top 5 venues of each neighborhood.

In [23]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = df.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

toronto_merged=toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,8,Other Great Outdoors,Health Food Store,Pub,Coffee Shop,Trail
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,8,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,8,Brewery,Burrito Place,Sushi Restaurant,Ice Cream Shop,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Distribution Center
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,8,Breakfast Spot,Park,Department Store,Food & Drink Shop,Hotel
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Sporting Goods Shop
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,8,Sandwich Place,Pizza Place,Dessert Shop,Sushi Restaurant,Café
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,8,Playground,Tennis Court,Park,Falafel Restaurant,Event Space
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,2,Coffee Shop,Pub,Bagel Shop,Restaurant,Supermarket


Visualizing the clusters:

In [24]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining the clusters

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,East Toronto,0,Café,Coffee Shop,Brewery,Gastropub,American Restaurant


In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
29,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Seafood Restaurant,American Restaurant


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Beer Bar,Japanese Restaurant


In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Central Toronto,4,Park,Swim School,Bus Line,Yoga Studio,Distribution Center


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Downtown Toronto,5,Coffee Shop,Italian Restaurant,Juice Bar,Japanese Restaurant,Burger Joint


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Central Toronto,6,Garden,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Downtown Toronto,7,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,8,Other Great Outdoors,Health Food Store,Pub,Coffee Shop,Trail
1,East Toronto,8,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop
2,East Toronto,8,Brewery,Burrito Place,Sushi Restaurant,Ice Cream Shop,Movie Theater
5,Central Toronto,8,Breakfast Spot,Park,Department Store,Food & Drink Shop,Hotel
7,Central Toronto,8,Sandwich Place,Pizza Place,Dessert Shop,Sushi Restaurant,Café
8,Central Toronto,8,Playground,Tennis Court,Park,Falafel Restaurant,Event Space
10,Downtown Toronto,8,Park,Playground,Trail,Dim Sum Restaurant,Falafel Restaurant
11,Downtown Toronto,8,Coffee Shop,Pizza Place,Bakery,Pub,Chinese Restaurant
14,Downtown Toronto,8,Coffee Shop,Clothing Store,Japanese Restaurant,Bubble Tea Shop,Café
15,Downtown Toronto,8,Café,Coffee Shop,Restaurant,Clothing Store,American Restaurant


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,Central Toronto,9,Sandwich Place,Café,Coffee Shop,Burger Joint,Park
